In [22]:
# import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import os

In [23]:
#load data
dataName="_mainExpAvDurEstimate_2025-03-27_15h13.32.171.csv"
#"_visualDurEstimate_2025-03-12_20h35.26.573.csv"

data = pd.read_csv("dataAvMain/"+dataName)
data[:4]
data['avgAVDeltaS'] = (data['deltaDurS'] + (data['recordedDurVisualTest'] - data['recordedDurVisualStandard'])) / 2
# Calculate deltaDurPercentVisual just as the difference between the test and standard visual durations over the standard visual duration
data['deltaDurPercentVisual'] = ((data['recordedDurVisualTest'] - data['recordedDurVisualStandard']) / data['recordedDurVisualStandard'] 
)

data['avgAVDeltaPercent'] = data[['delta_dur_percents', 'deltaDurPercentVisual']].mean(axis=1)
data

,Unnamed: 0,standardDur,riseDur,order,preDur,postDur,isiDur,trial_num,totalDur,delta_dur_percents,...,recordedOnsetVisualTest,recordedOffsetVisualTest,recordedDurVisualTest,recordedOnsetVisualStandard,recordedOffsetVisualStandard,recordedDurVisualStandard,modalityPostCue,avgAVDeltaS,deltaDurPercentVisual,avgAVDeltaPercent
0,0,0.4997,1.1993,2,0.4831,0.5414,0.5580,0,2.2071,-0.750,...,1.6528,1.7278,0.0750,0.4112,1.0527,0.6415,A,-0.47065,-0.883087,-0.816543
1,1,0.4997,0.0999,2,0.5414,0.4581,0.7163,1,2.3154,-0.800,...,1.8704,1.9204,0.0500,0.4703,1.1118,0.6415,A,-0.49565,-0.922058,-0.861029
2,2,0.4997,1.1993,1,0.6496,0.4498,0.6413,2,3.1149,0.750,...,0.6087,1.5748,0.9660,2.1745,2.7660,0.5915,V,0.37465,0.633136,0.691568
3,3,0.4997,0.0999,2,0.4914,0.5664,0.5664,3,3.0234,0.800,...,1.6694,2.5191,0.8497,0.4193,1.0612,0.6419,V,0.30380,0.323726,0.561863
4,4,0.4997,1.1993,1,0.5664,0.5997,0.6830,4,2.4737,-0.750,...,0.5227,0.7395,0.2168,1.3564,1.9980,0.6417,A,-0.39985,-0.662147,-0.706074
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
356,356,0.4997,1.1993,1,0.6163,0.5997,0.8162,356,3.4064,0.756,...,0.5708,1.5375,0.9667,2.3125,2.9041,0.5916,V,0.37495,0.634043,0.695022
357,357,0.4997,0.0999,2,0.5997,0.5247,0.7496,357,2.6902,-0.372,...,1.9661,2.2323,0.2662,0.5326,1.1743,0.6416,A,-0.27930,-0.585100,-0.478550
358,358,0.4997,0.0999,2,0.6080,0.4414,0.6413,358,2.9900,0.594,...,1.8655,2.6158,0.7503,0.5406,1.1825,0.6418,A,0.20415,0.169056,0.381528
359,359,0.4997,1.1993,1,0.5830,0.5330,0.8246,359,3.3398,0.800,...,0.5388,1.5300,0.9913,2.3133,2.9052,0.5918,V,0.39965,0.675059,0.737530


In [3]:
intensityVariable="avgAVDeltaPercent"

In [4]:
from scipy.stats import norm
from scipy.optimize import minimize

In [24]:
# parameters
# lambda_ fitted for all trials
# sigmaA sigmaB fitted for low and high noise conditions
# muA1, muA2, muA3, muB1, muB2, muB3 fitted for low and high noise conditions each for 3 different conflict levels
params = {
    'lambda_': 0.5,
    'sigmaA': 0.2,
    'sigmaB': 0.2,
    'muA1': 0.1,
    'muA2': 0.3,
    'muA3': 0.5,
    'muB1': 0.2,
    'muB2': 0.4,
    'muB3': 0.6
}
len(params)

9

In [6]:

def psychometric_function(intensities, lapse_rate, mu, sigma):
    # Cumulative distribution function with mean mu and standard deviation sigma
    cdf = norm.cdf(intensities, loc=mu, scale=sigma) 
    # take into account of lapse rate and return the probability of choosing test
    return lapse_rate * 0.5 + (1 - lapse_rate) * cdf 

def derivative_psychometric_function(intensities, lapse_rate, mu, sigma):
    #F'(x) = (1-lambda)*(1(/sqrt(2*pi)sigma)exp((x-mu)^2/sigma^2)

    return (1 - lapse_rate) * (1 / (np.sqrt(2 * np.pi) * sigma)) * np.exp(-((intensities - mu) ** 2) / (2 * sigma ** 2))    

# Negative log-likelihood
def negative_log_likelihood(params, delta_dur, chose_test, total_responses):
    
    lambda_, mu, sigma = params # Unpack parameters

    
    p = psychometric_function(delta_dur, lambda_, mu, sigma) # Compute probability of choosing test
    epsilon = 1e-9 # Add a small number to avoid log(0) when calculating the log-likelihood
    p = np.clip(p, epsilon, 1 - epsilon) # Clip p to avoid log(0) and log(1)
    # Compute the negative log-likelihood
    log_likelihood = np.sum(chose_test * np.log(p) + (total_responses - chose_test) * np.log(1 - p))
    return -log_likelihood


# Fit psychometric function
def fit_psychometric_function(levels,nResp, totalResp,init_guesses=[0,0,0]):
    # then fits the psychometric function
    # order is lambda mu sigma
    #initial_guess = [0, -0.2, 0.05]  # Initial guess for [lambda, mu, sigma]
    bounds = [(0, 0.2), (-0.4, +0.4), (0.01, 1)]  # Reasonable bounds
    # fitting is done here
    result = minimize(
        negative_log_likelihood, x0=init_guesses, 
        args=(levels, nResp, totalResp),  # Pass the data and fixed parameters
        bounds=bounds,
        method='Nelder-Mead'
    )
    # returns the fitted parameters lambda, mu, sigma
    return result.x

# Total NLL unified fitting



In [ ]:

def groupByChooseTest(x):
    grouped = x.groupby([intensityVariable, 'riseDur', 'standardDur','conflictDur']).agg(
        num_of_chose_test=('chose_test', 'sum'),
        total_responses=('responses', 'count'),
        num_of_chose_standard=('chose_standard', 'sum'),
    ).reset_index()
    grouped['p_choose_test'] = grouped['num_of_chose_test'] / grouped['total_responses']

    return grouped

In [8]:
def negative_log_likelihood(params, data):
    """
    params: [lambda_, sigmaA, sigmaB, muAn50, muA0, muAp50, muBn50, muB0, muBp50]
    data: DataFrame with all grouped trials
    """
    lambda_, sigmaA, sigmaB, muAn50, muA0, muAp50, muBn50, muB0, muBp50 = params

    nll = 0
    epsilon = 1e-9

    for _, row in data.iterrows():
        x = row[intensityVariable]  # intensity variable (e.g., avgAVDeltaPercent)
        k = row['num_of_chose_test']
        n = row['total_responses']
        conflict = round(row['conflictDur'], 2)
        audioNoise = round(row['riseDur'], 2)

        print(f"Processing: {x}, {k}, {n}, {conflict}, {audioNoise}")
        breakpoint()

        # Choose sigma
        if audioNoise == 0.1:
            sigma = sigmaA
            if conflict == -0.05:
                mu = muAn50
            elif conflict == 0:
                mu = muA0
            elif conflict == 0.05:
                mu = muAp50
        elif audioNoise == 1.2:
            sigma = sigmaB
            if conflict == -0.05:
                mu = muBn50
            elif conflict == 0:
                mu = muB0
            elif conflict == .05:
                mu = muBp50
        else:
            raise ValueError(f"Unknown audioNoise level: {audioNoise}")

        # Compute p
        p = psychometric_function(x, lambda_, mu, sigma)
        p = np.clip(p, epsilon, 1 - epsilon)
        nll -= k * np.log(p) + (n - k) * np.log(1 - p)

    return nll

from scipy.optimize import minimize

def fit_all_data(grouped_data):
    # Initial guesses: lambda, sigmaA, sigmaB, muAn50, muA0, muAp50, muBn50, muB0, muBp50
    init_guess = [0.03, 0.1, 0.1] + [0.0] * 6

    bounds = [(0, 0.2),      # lambda
            (0.02, 1),     # sigmaA
            (0.02, 1)] + [(-0.4, 0.4)] * 6  # all mu bounds

    result = minimize(
        negative_log_likelihood,
        x0=init_guess,
        args=(grouped_data,),
        bounds=bounds,
        method='Nelder-Mead'
    )
    return result


In [19]:
data

,Unnamed: 0,standardDur,riseDur,order,preDur,postDur,isiDur,trial_num,totalDur,delta_dur_percents,...,recordedOnsetVisualTest,recordedOffsetVisualTest,recordedDurVisualTest,recordedOnsetVisualStandard,recordedOffsetVisualStandard,recordedDurVisualStandard,modalityPostCue,avgAVDeltaS,deltaDurPercentVisual,avgAVDeltaPercent
0,0,0.4997,1.1993,2,0.4831,0.5414,0.5580,0,2.2071,-0.750,...,1.6528,1.7278,0.0750,0.4112,1.0527,0.6415,A,-0.47065,-0.883087,-0.816543
1,1,0.4997,0.0999,2,0.5414,0.4581,0.7163,1,2.3154,-0.800,...,1.8704,1.9204,0.0500,0.4703,1.1118,0.6415,A,-0.49565,-0.922058,-0.861029
2,2,0.4997,1.1993,1,0.6496,0.4498,0.6413,2,3.1149,0.750,...,0.6087,1.5748,0.9660,2.1745,2.7660,0.5915,V,0.37465,0.633136,0.691568
3,3,0.4997,0.0999,2,0.4914,0.5664,0.5664,3,3.0234,0.800,...,1.6694,2.5191,0.8497,0.4193,1.0612,0.6419,V,0.30380,0.323726,0.561863
4,4,0.4997,1.1993,1,0.5664,0.5997,0.6830,4,2.4737,-0.750,...,0.5227,0.7395,0.2168,1.3564,1.9980,0.6417,A,-0.39985,-0.662147,-0.706074
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
356,356,0.4997,1.1993,1,0.6163,0.5997,0.8162,356,3.4064,0.756,...,0.5708,1.5375,0.9667,2.3125,2.9041,0.5916,V,0.37495,0.634043,0.695022
357,357,0.4997,0.0999,2,0.5997,0.5247,0.7496,357,2.6902,-0.372,...,1.9661,2.2323,0.2662,0.5326,1.1743,0.6416,A,-0.27930,-0.585100,-0.478550
358,358,0.4997,0.0999,2,0.6080,0.4414,0.6413,358,2.9900,0.594,...,1.8655,2.6158,0.7503,0.5406,1.1825,0.6418,A,0.20415,0.169056,0.381528
359,359,0.4997,1.1993,1,0.5830,0.5330,0.8246,359,3.3398,0.800,...,0.5388,1.5300,0.9913,2.3133,2.9052,0.5918,V,0.39965,0.675059,0.737530


In [18]:
grouped_data = groupByChooseTest(data)  # or your own grouping
result = fit_all_data(grouped_data)
params = result.x

lambda_ = params[0]
sigmaA = params[1]
sigmaB = params[2]
mu_labels = ["muA0", "muA1", "muA2", "muB0", "muB1", "muB2"]
mus = dict(zip(mu_labels, params[3:]))

print("Lambda:", lambda_)
print("Sigma A/B:", sigmaA, sigmaB)
for k, v in mus.items():
    print(f"{k}: {v:.3f}")


KeyError: "Column(s) ['chose_standard', 'chose_test'] do not exist"

In [12]:


# Plotting the fitted psychometric function
def plot_psychometric_function(intensities, lambda_, mus, sigmaA, sigmaB):
    plt.figure(figsize=(10, 6))
    for i in range(3):
        mu = mus[f"muA{i}"]
        sigma = sigmaA
        p = psychometric_function(intensities, lambda_, mu, sigma)
        plt.plot(intensities, p, label=f"Audio Noise: 0.1, Conflict: {i}")

    for i in range(3):
        mu = mus[f"muB{i}"]
        sigma = sigmaB
        p = psychometric_function(intensities, lambda_, mu, sigma)
        plt.plot(intensities, p, label=f"Audio Noise: 1.2, Conflict: {i}")

    plt.xlabel("Intensity")
    plt.ylabel("Probability of Choosing Test")
    plt.title("Psychometric Function")
    plt.legend()
    plt.grid()
    plt.show()

# Generate a range of intensities for plotting
intensities = np.linspace(-0.5, 0.5, 100)
plot_psychometric_function(intensities, lambda_, mus, sigmaA, sigmaB)
def groupByChooseTest(data):
    # Group by intensity variable and calculate the number of responses
    grouped_data = data.groupby(intensityVariable).agg(
        num_of_chose_test=('chose_test', 'sum'),
        total_responses=('chose_test', 'count'),
        conflictDur=('conflictDur', 'first'),
        riseDur=('riseDur', 'first')
    ).reset_index()
    return grouped_data
def plotPsychometric(data):
    # Group by intensity variable and calculate the number of responses
    grouped_data = data.groupby(intensityVariable).agg(
        num_of_chose_test=('chose_test', 'sum'),
        total_responses=('chose_test', 'count'),
        conflictDur=('conflictDur', 'first'),
        riseDur=('riseDur', 'first')
    ).reset_index()

    # Plotting
    plt.figure(figsize=(10, 6))
    sns.scatterplot(data=grouped_data, x=intensityVariable, y='num_of_chose_test', size='total_responses', sizes=(20, 200), alpha=0.5)
    plt.xlabel(intensityVariable)
    plt.ylabel('Number of Chose Test')
    plt.title('Psychometric Function Data')
    plt.grid()
    plt.show()
plotPsychometric(data)

NameError: name 'lambda_' is not defined

In [ ]:

# Compute sigma from slope
def compute_sigma_from_slope(slope, lapse_rate=0.02):
    sigma = (1 - lapse_rate) / (np.sqrt(2 * np.pi) * slope)*np.exp(-0.5)
    return sigma


def fitMultipleStartingPoints(levels, nResp, totalResp, multipleInitGuesses, fixedLapse=None,fixedSigma=None):
    best_fit = None
    best_nll = float('inf')  # Initialize with infinity

    for init_guesses in multipleInitGuesses:
        try:
            fit = fit_psychometric_function(levels, nResp, totalResp, init_guesses, fixedLapse, fixedSigma)
            nll = negative_log_likelihood(fit, levels, nResp, totalResp,fixedLapse,fixedSigma)

            if nll < best_nll:
                best_nll = nll
                best_fit = fit
        except Exception as e:
            print(f"Error fitting with initial guesses {init_guesses}: {e}")

    return best_fit



In [ ]:
def get_mu_index(conflict_level, noise_level, conflict_levels, noise_levels):
    conflict_idx = conflict_levels.index(conflict_level)
    noise_idx = noise_levels.index(noise_level)
    return 3 * noise_idx + conflict_idx  # maps to mu_1A to mu_3B


In [26]:
# Prepare dataframe for fitting

def prepare_data(data, intensityVariable):
    modelMatrix = pd.DataFrame()
    # Prepare data for fitting
    modelMatrix["standardDur"] = data['standardDur']
    modelMatrix['audioNoise']=data['riseDur']
    modelMatrix["conflictLevel"]=data['conflictDur']
    modelMatrix['intensity'] = data[intensityVariable]
    modelMatrix["choseTest"]=data['chose_test'].astype(int)
    modelMatrix['lambda']=''

    return modelMatrix


modelMatrix=prepare_data(data, intensityVariable)
modelMatrix



KeyError: 'chose_test'

,Unnamed: 0,standardDur,riseDur,order,preDur,postDur,isiDur,trial_num,totalDur,delta_dur_percents,...,recordedDurVisualStandard,modalityPostCue,avgAVDeltaS,deltaDurPercentVisual,avgAVDeltaPercent,chose_test,chose_standard,standard_dur,intensity,audioNoise
0,0,0.4997,1.1993,2,0.4831,0.5414,0.5580,0,2.2071,-0.750,...,0.6415,A,-0.47065,-0.883087,-0.816543,0,1,0.4997,-0.816543,1.1993
1,1,0.4997,0.0999,2,0.5414,0.4581,0.7163,1,2.3154,-0.800,...,0.6415,A,-0.49565,-0.922058,-0.861029,0,1,0.4997,-0.861029,0.0999
2,2,0.4997,1.1993,1,0.6496,0.4498,0.6413,2,3.1149,0.750,...,0.5915,V,0.37465,0.633136,0.691568,1,0,0.4997,0.691568,1.1993
3,3,0.4997,0.0999,2,0.4914,0.5664,0.5664,3,3.0234,0.800,...,0.6419,V,0.30380,0.323726,0.561863,1,0,0.4997,0.561863,0.0999
4,4,0.4997,1.1993,1,0.5664,0.5997,0.6830,4,2.4737,-0.750,...,0.6417,A,-0.39985,-0.662147,-0.706074,0,1,0.4997,-0.706074,1.1993
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
356,356,0.4997,1.1993,1,0.6163,0.5997,0.8162,356,3.4064,0.756,...,0.5916,V,0.37495,0.634043,0.695022,0,1,0.4997,0.695022,1.1993
357,357,0.4997,0.0999,2,0.5997,0.5247,0.7496,357,2.6902,-0.372,...,0.6416,A,-0.27930,-0.585100,-0.478550,0,1,0.4997,-0.478550,0.0999
358,358,0.4997,0.0999,2,0.6080,0.4414,0.6413,358,2.9900,0.594,...,0.6418,A,0.20415,0.169056,0.381528,1,0,0.4997,0.381528,0.0999
359,359,0.4997,1.1993,1,0.5830,0.5330,0.8246,359,3.3398,0.800,...,0.5918,V,0.39965,0.675059,0.737530,0,1,0.4997,0.737530,1.1993


In [ ]:
# Define columns for chosing test or standard
data['chose_test'] = (data['responses'] == data['order']).astype(int)
data['chose_standard'] = (data['responses'] != data['order']).astype(int)
try:
    print(data["riseDur"]>1)
except:
    data["riseDur"]=1

data['standard_dur']=data['standardDur']

def groupByChooseTest(x):
    grouped = x.groupby([intensityVariable, 'riseDur', 'standard_dur','conflictDur']).agg(
        num_of_chose_test=('chose_test', 'sum'),
        total_responses=('responses', 'count'),
        num_of_chose_standard=('chose_standard', 'sum'),
    ).reset_index()
    grouped['p_choose_test'] = grouped['num_of_chose_test'] / grouped['total_responses']

    return grouped

def groupByStandardDur(x):
    grouped = x.groupby([intensityVariable, 'riseDur', 'standard_dur']).agg(
        num_of_chose_test=('chose_test', 'sum'),
        total_responses=('responses', 'count'),
        num_of_chose_standard=('chose_standard', 'sum')
    ).reset_index()
    grouped['pChooseStandard'] = grouped['num_of_chose_standard'] / grouped['total_responses']

    return grouped

grouped=groupByChooseTest(data)
# p_choose_test
#sort the group
grouped = grouped.sort_values([ 'standard_dur'])

0       True
1      False
2       True
3      False
4       True
       ...  
356     True
357    False
358    False
359     True
360    False
Name: riseDur, Length: 361, dtype: bool


In [ ]:
grouped

,avgAVDeltaPercent,riseDur,standard_dur,conflictDur,num_of_chose_test,total_responses,num_of_chose_standard,p_choose_test
0,-0.950000,0.0999,0.4997,-0.05,0,1,1,0.0
242,0.520456,0.0999,0.4997,-0.05,1,1,0,1.0
241,0.520223,0.0999,0.4997,0.05,1,1,0,1.0
240,0.517824,0.0999,0.4997,-0.05,1,1,0,1.0
239,0.517534,0.0999,0.4997,0.05,1,1,0,1.0
...,...,...,...,...,...,...,...,...
114,-0.450405,0.0999,0.4997,0.05,1,1,0,1.0
113,-0.466996,1.1993,0.4997,-0.05,0,1,1,0.0
112,-0.467251,0.0999,0.4997,-0.05,0,1,1,0.0
132,-0.396116,1.1993,0.4997,0.00,0,2,2,0.0


In [ ]:
conflictLevels=grouped['conflictDur'].unique()
noiseLevels=grouped['riseDur'].unique()

In [ ]:
from scipy.optimize import minimize

# Initial guess: lambda, sigmaA, sigmaB, 6 mus
init_guess = [0.03, 0.1, 0.1] + [0.0] * 6
bounds = [(0, 0.2), (0.01, 1), (0.01, 1)] + [(-0.4, 0.4)] * 6

result = minimize(
    total_negative_log_likelihood,
    x0=init_guess,
    args=(grouped, conflictLevels, noiseLevels),
    bounds=bounds,
    method='Nelder-Mead'  # or try L-BFGS-B if faster
)


KeyError: 'delta_dur_percents'

In [ ]:
def group_data(df):
    grouped = df.groupby(['delta_dur_percents', 'conflictDur', 'riseDur']).agg(
        num_of_chose_test=('choseTest', 'sum'),
        total_responses=('choseTest', 'count')
    ).reset_index()
    return grouped
grouped_data = group_data(data)
grouped_data.columns
# ['delta_dur_percents', 'num_of_chose_test', 'total_responses', 'conflictDur', 'riseDur']


Index(['delta_dur_percents', 'conflictDur', 'riseDur', 'num_of_chose_test',
       'total_responses'],
      dtype='object')

In [ ]:
from scipy.stats import norm

def psychometric_function(intensities, lapse_rate, mu, sigma):
    cdf = norm.cdf(intensities, loc=mu, scale=sigma)
    return lapse_rate * 0.5 + (1 - lapse_rate) * cdf


In [ ]:
def total_negative_log_likelihood(params, data, conflict_levels, noise_levels):
    """
    params = [lambda, sigma_A, sigma_B, mu_1A, mu_2A, mu_3A, mu_1B, mu_2B, mu_3B]
    """
    lambda_ = params[0]
    sigma_dict = {
        noise_levels[0]: params[1],  # sigma_A
        noise_levels[1]: params[2],  # sigma_B
    }

    # Unpack the mus (biases) in a dictionary keyed by (conflict, noise)
    mu_values = params[3:]  # 6 mus
    mu_dict = {}
    idx = 0
    for noise in noise_levels:
        for conflict in conflict_levels:
            mu_dict[(conflict, noise)] = mu_values[idx]
            idx += 1

    nll = 0
    epsilon = 1e-9
    for _, row in data.iterrows():
        x = row["delta_dur_percents"]
        k = row["num_of_chose_test"]
        n = row["total_responses"]
        conflict = row["conflictDur"]
        noise = row["riseDur"]

        mu = mu_dict[(conflict, noise)]
        sigma = sigma_dict[noise]

        p = psychometric_function(x, lambda_, mu, sigma)
        p = np.clip(p, epsilon, 1 - epsilon)
        nll -= k * np.log(p) + (n - k) * np.log(1 - p)

    return nll


In [29]:
from scipy.optimize import minimize

# Example: If your conflict levels are [0, 50, -50] and noise levels are [0.1, 0.2]
conflict_levels = [0, 50, -50]
noise_levels = [0.1, 0.2]

# Initial guess: [lambda, sigmaA, sigmaB, mu_1A, mu_2A, mu_3A, mu_1B, mu_2B, mu_3B]
init_guess = [0.03, 0.1, 0.1] + [0.0] * 6

# Bounds for each param
bounds = [(0, 0.2), (0.02, 1.0), (0.02, 1.0)] + [(-0.4, 0.4)] * 6

# Fit using all grouped data
# Map conflict and noise levels in grouped_data to match conflict_levels and noise_levels
grouped_data['conflictDur'] = grouped_data['conflictDur'].round(2).map({-0.05: -50, 0.05: 50, 0.00: 0})
grouped_data['riseDur'] = grouped_data['riseDur'].round(1).map({0.1: 0.1, 1.2: 1.2})

# Drop rows with unmapped values (if any)
grouped_data = grouped_data.dropna(subset=['conflictDur', 'riseDur'])

result = minimize(
    total_negative_log_likelihood,
    x0=init_guess,
    args=(grouped_data, conflict_levels, noise_levels),
    bounds=bounds,
    method='Nelder-Mead'  # or try L-BFGS-B for better performance
)


NameError: name 'grouped_data' is not defined

In [30]:
lambda_fit = result.x[0]
sigma_A, sigma_B = result.x[1:3]
mu_dict = {}

idx = 3
for noise in noise_levels:
    for conflict in conflict_levels:
        mu_dict[(conflict, noise)] = result.x[idx]
        idx += 1


NameError: name 'result' is not defined

In [31]:
x = np.linspace(-1, 1, 100)
for noise in noise_levels:
    for conflict in conflict_levels:
        mu = mu_dict[(conflict, noise)]
        sigma = sigma_A if noise == noise_levels[0] else sigma_B
        y = psychometric_function(x, lambda_fit, mu, sigma)
        plt.plot(x, y, label=f"conf={conflict}, noise={noise}")
plt.legend()
plt.title("Fitted Psychometric Curves")
plt.xlabel("Δ duration")
plt.ylabel("P(choose test)")
plt.grid()
plt.show()


NameError: name 'mu_dict' is not defined